In [2]:
import spoti
from requests import get, post
import json
import webbrowser
import requests
import base64
import os
import urllib.parse
import pandas as pd
import time

Python-dotenv could not parse statement starting at line 3


In [3]:
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
redirect_uri = "http://localhost:8501/callback"

api_token = spoti.get_token()
scope = ''

In [4]:
def get_authorization_code():
    auth_url = (
        "https://accounts.spotify.com/authorize"
        "?response_type=code"
        f"&client_id={client_id}"
        f"&scope={scope}"
        f"&redirect_uri={urllib.parse.quote(redirect_uri, safe='')}"
    )

    webbrowser.open(auth_url)

    return input("Paste the full redirect URL here: ")


def extract_auth_code(redirect_url):
    parsed_url = urllib.parse.urlparse(redirect_url)
    auth_code = urllib.parse.parse_qs(parsed_url.query).get("code", [None])[0]
    
    return auth_code


def get_access_token(auth_code):
    token_url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    headers = {
        "Authorization": f"Basic {auth_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_url, headers=headers, data=data)
    response_json = response.json()

    return response_json["access_token"]


In [5]:
def save_to_csv(recent_tracks, filename):
      
    for track in recent_tracks:
        track["artist_names"] = ", ".join([artist["artist_name"] for artist in track["artists"]])
        track["artist_ids"] = ", ".join([artist["artist_id"] for artist in track["artists"]])
        del track["artists"]
    
    df = pd.DataFrame(recent_tracks)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print('Data saved to CSV file')

### Make Database of Listening History from Recents

In [12]:
def recently_played(access_token):
    url = "https://api.spotify.com/v1/me/player/recently-played?limit=50"
    headers = {"Authorization": f"Bearer {access_token}"}

    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    
    track_info = []
    
    for item in json_result["items"]:
        
        track = item["track"]
        info = {
            "track_name": track["name"],
            "track_id": track["id"],
            "album_name": track["album"]["name"],
            "album_id": track["album"]["id"],
            "album_release_date": track["album"]["release_date"],
            "popularity": track["popularity"],
            "artists": [
                {"artist_name": artist["name"], "artist_id": artist["id"]}
                for artist in track["artists"]
            ],
        }
        
        track_info.append(info)
    
    return track_info


scope = "user-read-recently-played"

redirect_url = get_authorization_code()
auth_code = extract_auth_code(redirect_url)
access_token = get_access_token(auth_code)
print(f"✅ Access Token")

recent_tracks = recently_played(access_token)
save_to_csv(recent_tracks, filename="new_recents.csv")

✅ Access Token
Data saved to CSV file


In [13]:
full_recents = pd.read_csv('recent_tracks.csv', encoding='utf-8')
new_recents = pd.read_csv('new_recents.csv', encoding='utf-8')

recents_df = pd.concat([full_recents, new_recents], ignore_index=True)
recents_df.drop_duplicates(keep='last', inplace=True)
recents_df.to_csv('recent_tracks.csv', index=False, encoding='utf-8')

### Fetch All Liked Songs

In [ ]:
def liked_songs(token):
    
    url = 'https://api.spotify.com/v1/me/tracks?limit=50'
    headers = {'Authorization': f'Bearer {token}'}
    
    all_likes = []
    
    while url:
        
        result = get(url, headers=headers)
        json_result = json.loads(result.content)
        
        for item in json_result["items"]:
            
            track = item["track"]
            info = {
                "track_name": track["name"],
                "track_id": track["id"],
                "album_name": track["album"]["name"],
                "album_id": track["album"]["id"],
                "album_release_date": track["album"]["release_date"],
                "popularity": track["popularity"],
                "artists": [
                    {"artist_name": artist["name"], "artist_id": artist["id"]}
                    for artist in track["artists"]
                ],
            }
            all_likes.append(info)
            
        url = json_result["next"]
    
    return all_likes


scope = "user-library-read"

redirect_url = get_authorization_code()
auth_code = extract_auth_code(redirect_url)
access_token = get_access_token(auth_code)
print(f"✅ Access Token: {access_token}")

liked_songs = liked_songs(access_token)
# save_to_csv(liked_songs, filename="liked_songs.csv")

### Create Database of Liked Songs per Album

In [ ]:
df = pd.read_csv('Music Data/Spotipy/liked_songs.csv')
album_wise_liked = []

for album in df['album_name'].unique():
    liked_album_data = df.loc[df['album_name']==album]
    
    liked_popularity = liked_album_data['popularity'].mean()
    album_liked = ""
    
    for song in liked_album_data['track_name']:
        album_liked += song + ', '
        
    entry = {
        "music project": album,
        "liked songs": album_liked.rstrip(', '),
        "liked popularity": round(liked_popularity, 2)
    }
    album_wise_liked.append(entry)
    
album_wise_liked = pd.DataFrame(album_wise_liked)
album_wise_liked.to_csv('Music Data/album_wise_liked.csv', encoding='utf-8')
album_wise_liked.head(10)

,music project,liked songs,liked popularity
0,Jinx,Ghostride,72.00
1,MUSIC,"OVERLY, HBA, WALK, OPM BABI, OLYMPIAN, COCAINE...",83.81
2,AT.LONG.LAST.A$AP,"M'$ (feat. Lil Wayne), Better Things, Wavybone...",63.08
3,Sensational,"1 Thot 2 Thot Red Thot Blue Thot, Whip A Tesla...",50.67
4,Yasuke,"Between Memories, The Eyes of Vengeance, Your ...",33.14
5,Arbi Ghode,Arbi Ghode,57.00
6,"Mancini Plays the Theme from ""Love Story""","Theme from ""Love Story""",34.00
7,Make Way for Dionne Warwick,Walk on By,67.00
8,Evr’y Time We Say Goodbye - All-Time Romantic ...,Georgia on My Mind,56.00
9,The Genesis of Aretha: 1960-1966,One Step Ahead,47.00


In [ ]:
og_df = pd.read_excel('Music Data/my_music_history.xlsx', encoding='utf-8')
og_df.head()

,artist,music project,spotify logged,runtime,release date,genre tags
0,Travis Scott,Birds in the Trap Sing McKnight,Yes,"14 tracks, 53:38",2 September 2016,"['hip-hop', 'trap', '2016', 'rap', 'pop rap']"
1,Frank Ocean,Blonde,Yes,"17 tracks, 64:51",20 August 2016,"['alternative rnb', 'art pop', 'neo-soul', '20..."
2,A$AP Rocky,LONG.LIVE.A$AP,Yes,17 tracks,10 January 2013,"['rap', 'hip-hop', 'cloud rap', 'east coast hi..."
3,Kanye West,ye [Explicit],Yes,7 tracks,5 November 2018,"['rap', 'hip-hop', 'great', 'spoken word', 'ka..."
4,Freddie Gibbs,Bandana,Yes,NaN,NaN,"['2019', 'rap', 'hip hop', 'gangsta rap', 'fre..."


In [13]:
result = pd.merge(og_df, album_wise_liked, on='music project', how='left')
result.to_csv('Music Data/full_dataset.csv', encoding='utf-8')
result.head(20)

,artist,music project,spotify logged,runtime,release date,genre tags,liked songs,liked popularity
0,Travis Scott,Birds in the Trap Sing McKnight,Yes,"14 tracks, 53:38",2 September 2016,"['hip-hop', 'trap', '2016', 'rap', 'pop rap']",NaN,NaN
1,Frank Ocean,Blonde,Yes,"17 tracks, 64:51",20 August 2016,"['alternative rnb', 'art pop', 'neo-soul', '20...","Pink + White, Ivy, Nights",84.67
2,A$AP Rocky,LONG.LIVE.A$AP,Yes,17 tracks,10 January 2013,"['rap', 'hip-hop', 'cloud rap', 'east coast hi...",NaN,NaN
3,Kanye West,ye [Explicit],Yes,7 tracks,5 November 2018,"['rap', 'hip-hop', 'great', 'spoken word', 'ka...",NaN,NaN
4,Freddie Gibbs,Bandana,Yes,NaN,NaN,"['2019', 'rap', 'hip hop', 'gangsta rap', 'fre...","Bandana, Crime Pays, Freestyle Shit, Gat Damn,...",48.33
5,Kanye West,Yeezus,Yes,"10 tracks, 37:12",1 January 2013,"['rap', 'hip-hop', 'hip hop', 'uh huh honey', ...","Bound 2, Blood On The Leaves, I'm In It, Hold ...",67.38
6,Error fetch,Graduation,No,"13 tracks, 47:57",1 January 2007,"['hip-hop', 'rap', 'kanye west', 'hip hop', 'r...","Good Life, Flashing Lights, Homecoming, Drunk ...",10.60
7,Joey Bada$$,B4.DA.$$,Yes,"17 tracks, 61:31",19 January 2015,"['2015', 'jazz rap', 'hip-hop', 'boom bap', 'e...","Save The Children, Greenbax (Introlude), Paper...",37.79
8,Joey Bada$$,ALL-AMERIKKKAN BADA$$,Yes,"12 tracks, 49:42",7 April 2017,"['hip-hop', '2017', 'rap', 'east coast hip hop...","DEVASTATED, ROCKABYE BABY, LEGENDARY",54.33
9,Lil' Wayne,Tha Carter V,Yes,"23 tracks, 87:43",26 September 2018,"['hip-hop', 'rap', 'lil wayne', 'metro boomin'...","Dedicate, Let It All Work Out, Mona Lisa (feat...",68.33
